For study -  
512 color image public eff7  
384 gray image size -> 1.25 size up (37.9)   with softmax(주의해야 함, logit이 sigmoid에 최적화)
480 gray image size -> random sizeup (1,1.1) (37.6 map)  (주의해야 함, logit이 sigmoid에 최적화)
(two 380, 480 ensemble -> 38.5)  

Two class  
512 color image size ->public eff7 (0.87 auc)  

Object deteciton  
640 gray image size -> yolov5 M -> (0.523)

Two class detection
384 gray image size -> 1.25 size up   with softmax(주의해야 함, logit이 sigmoid에 최적화)  
480 gray image size -> random sizeup (1,1.1)
Two class auc 89.2  

Object detection에 bayesian 확률을 활용해서 더 현실적으로 만들어야 함  
384, 480, 440, 540, 640 random(1,1.33)에서 cv 최적화(39.3)

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y
!pip install ../input/timm-pytorch-image-models/pytorch-image-models-master

!pip install '/kaggle/input/kerasapplications' --no-deps
!pip install '/kaggle/input/efficientnet-keras-source-code' --no-deps
#!pip install '/kaggle/input/effdet-latestvinbigdata-wbf-fused/ensemble_boxes-1.0.4-py3-none-any.whl' --no-deps

In [ ]:
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchvision-0.8.1+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchaudio-0.7.0-cp37-cp37m-linux_x86_64.whl' --no-deps

## Compatible Cuda Toolkit installation
!mkdir -p /kaggle/tmp && cp /kaggle/input/pytorch-170-cuda-toolkit-110221/cudatoolkit-11.0.221-h6bb024c_0 /kaggle/tmp/cudatoolkit-11.0.221-h6bb024c_0.tar.bz2 && conda install /kaggle/tmp/cudatoolkit-11.0.221-h6bb024c_0.tar.bz2 -y --offline

## MMDetection Offline Installation
!pip install '/kaggle/input/mmdetectionv2140/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmcv_full-1_3_8-cu110-torch1_7_0/mmcv_full-1.3.8-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps

!cp -r /kaggle/input/mmdetectionv2140/mmdetection-2.14.0 /kaggle/working/
!mv /kaggle/working/mmdetection-2.14.0 /kaggle/working/mmdetection
%cd /kaggle/working/mmdetection
!pip install -e . --no-deps
%cd /kaggle/working/

In [ ]:
study_torch_384size = 384
study_torch_480size = 480
study_torch_440size = 440
study_torch_540size = 540
study_torch_640size = 640
study_torch_size = 640
study_tf_size = 512
image_size = 640
twoclass_size = 512
twoclass_480size = 480
twoclass_384size = 384

effv_model_path_384 = "../input/onecycle-effv2-384/"
effv_model_path_480 = "../input/480imageeffv2cv376/"
effv_large_model_path_440 = "../input/onecycle-effv2-large-440/"
effv_large_model_path_540 = "../input/onecycle-effv2-large-540/"
effv_large_model_path_640 = "../input/effvlarge640/"
effv_model_twoclass_384 = "../input/efftwoclass384/"
effv_model_twoclass_480 = "../input/efftwoclass480/"

In [ ]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
if df.shape[0] == 2477:
    fast_sub = True
    fast_df = pd.DataFrame(([['00086460a852_study', 'negative 1 0 0 1 1'], 
                         ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
                         ['65761e66de9f_image', 'none 1 0 0 1 1'], 
                         ['51759b5579bc_image', 'none 1 0 0 1 1']]), 
                       columns=['id', 'PredictionString'])
else:
    fast_sub = False

# .dcm to .png

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
split = 'test'
save_dir = f'/kaggle/tmp/{split}/'

os.makedirs(save_dir, exist_ok=True)

save_dir = f'/kaggle/tmp/{split}/study_torch_size/'
os.makedirs(save_dir, exist_ok=True)
if fast_sub:
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=study_torch_size)  
    study = '00086460a852' + '_study.png'
    im.save(os.path.join(save_dir, study))
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    #print(xray.shape)
    im = resize(xray, size=study_torch_size)  
    study = '000c9c05fd14' + '_study.png'
    im.save(os.path.join(save_dir, study))
else:   
    for dirname, _, filenames in tqdm(os.walk(f'/kaggle/input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=study_torch_size)  
            study = dirname.split('/')[-2] + '_study.png'
            im.save(os.path.join(save_dir, study))

In [ ]:
image_id = []
dim0 = []
dim1 = []
splits = []
save_dir = f'/kaggle/tmp/{split}/image/'
os.makedirs(save_dir, exist_ok=True)
if fast_sub:
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=image_size)  
    im.save(os.path.join(save_dir,'65761e66de9f_image.png'))
    image_id.append('65761e66de9f.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
    xray = read_xray('/kaggle/input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=image_size)  
    im.save(os.path.join(save_dir, '51759b5579bc_image.png'))
    image_id.append('51759b5579bc.dcm'.replace('.dcm', ''))
    dim0.append(xray.shape[0])
    dim1.append(xray.shape[1])
    splits.append(split)
else:
    for dirname, _, filenames in tqdm(os.walk(f'/kaggle/input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=image_size)  
            im.save(os.path.join(save_dir, file.replace('.dcm', '_image.png')))
            image_id.append(file.replace('.dcm', ''))
            dim0.append(xray.shape[0])
            dim1.append(xray.shape[1])
            splits.append(split)
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})

# study predict

In [ ]:
import numpy as np 
import pandas as pd
if fast_sub:
    df = fast_df.copy()
else:
    df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
id_laststr_list  = []
for i in range(df.shape[0]):
    id_laststr_list.append(df.loc[i,'id'][-1])
df['id_last_str'] = id_laststr_list

study_len = df[df['id_last_str'] == 'y'].shape[0]

# For Pytorch Markdown

In [ ]:
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
sub_df = sub_df[:study_len]
test_paths = f'/kaggle/tmp/{split}/study_torch/' + sub_df['id'] +'.png'

sub_df['negative'] = 0
sub_df['typical'] = 0
sub_df['indeterminate'] = 0
sub_df['atypical'] = 0


label_cols = sub_df.columns[2:]

# Pytorch ImageLoader

In [ ]:
# torch libs
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parallel.data_parallel import data_parallel

from torch.nn.utils.rnn import *


def seed_torch(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

import os
from datetime import datetime

#-----------------------------------

#numerical libs
import math
import numpy as np
import random
import PIL
import cv2
import matplotlib
#matplotlib.use('TkAgg')
#matplotlib.use('WXAgg')
#matplotlib.use('Qt4Agg')
#matplotlib.use('Qt5Agg')
#print('matplotlib.get_backend : ', matplotlib.get_backend())
#print(matplotlib.__version__)


# std libs
import collections
from collections import defaultdict
import copy
import numbers
import inspect
import shutil
from timeit import default_timer as timer
import itertools
from collections import OrderedDict
from multiprocessing import Pool
import multiprocessing as mp

#from pprintpp import pprint, pformat
import json
import zipfile
from shutil import copyfile

import csv
import pandas as pd
import pickle
import glob
import sys
from distutils.dir_util import copy_tree
import time

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def null_augment(r):
    image = r['image']
    # if image[:2].shape != (image_size, image_size):
    #     r['image'] = cv2.resize(image, dsize=(image_size, image_size), interpolation=cv2.INTER_AREA)
    return r


class SiimDataset(Dataset):
    def __init__(self, df, image_size, augment=null_augment):
        super().__init__()
        self.df = df
        self.augment = augment
        self.length = len(df)
        self.image_size = image_size

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        d = self.df.iloc[index]

        #image_file = data_dir + '/%s_640/%s/%s/%s.png' % (d.set, d.study, d.series, d.image)
        #image_file = data_dir + '/%s_full_512/%s.png' % (d.set, d.image)
        image_file = f'/kaggle/tmp/{split}/study_torch_size/' + d['id'].split("_")[0] + "_study.png"
        image = cv2.imread(image_file,cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.image_size, self.image_size))

        r = {'image' : image, 'image_size' : self.image_size}
        if self.augment is not None: r = self.augment(r)
        return r
    
def null_collate(batch):
    collate = defaultdict(list)

    for r in batch:
        for k, v in r.items():
            collate[k].append(v)

    # ---
    batch_size = len(batch)
    image = np.stack(collate['image'])
    image = image.reshape(batch_size, 1, collate['image_size'][0],collate['image_size'][0]).repeat(3,1)
    image = np.ascontiguousarray(image)
    image = image.astype(np.float32) / 255
    collate['image'] = torch.from_numpy(image)

    return collate

test_384dataset = SiimDataset(sub_df, study_torch_384size)
test_384loader  = DataLoader(
                test_384dataset,
                sampler = SequentialSampler(test_384dataset),
                batch_size  = 32,#128, #
                drop_last   = False,
                num_workers = 0,
                pin_memory  = True,
                collate_fn  = null_collate,
            )

test_480dataset = SiimDataset(sub_df, study_torch_480size)
test_480loader  = DataLoader(
                test_480dataset,
                sampler = SequentialSampler(test_480dataset),
                batch_size  = 32,#128, #
                drop_last   = False,
                num_workers = 0,
                pin_memory  = True,
                collate_fn  = null_collate,
            )

test_440dataset = SiimDataset(sub_df, study_torch_440size)
test_440loader  = DataLoader(
                test_440dataset,
                sampler = SequentialSampler(test_440dataset),
                batch_size  = 32,#128, #
                drop_last   = False,
                num_workers = 0,
                pin_memory  = True,
                collate_fn  = null_collate,
            )

test_540dataset = SiimDataset(sub_df, study_torch_540size)
test_540loader  = DataLoader(
                test_540dataset,
                sampler = SequentialSampler(test_540dataset),
                batch_size  = 16,#128, #
                drop_last   = False,
                num_workers = 0,
                pin_memory  = True,
                collate_fn  = null_collate,
            )


# Torch NetWork

In [ ]:
sys.path.append("../input/timm-pytorch-image-models/pytorch-image-models-master/")
from timm.models.efficientnet import *


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        e = tf_efficientnetv2_m(pretrained=False)
        self.b0 = nn.Sequential(
            e.conv_stem,
            e.bn1,
            e.act1,
        )
        self.b1 = e.blocks[0]
        self.b2 = e.blocks[1]
        self.b3 = e.blocks[2]
        self.b4 = e.blocks[3]
        self.b5 = e.blocks[4]
        self.b6 = e.blocks[5]
        self.b7 = e.blocks[6]
        self.b8 = nn.Sequential(
            e.conv_head, #384, 1536
            e.bn2,
            e.act2,
        )

        self.logit = nn.Linear(1280,4)
        self.mask = nn.Sequential(
            nn.Conv2d(176, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    # @torch.cuda.amp.autocast()
    def forward(self, image):
        batch_size = len(image)
        x = 2*image-1     # ; print('input ',   x.shape)

        x = self.b0(x) #; print (x.shape)  # torch.Size([2, 40, 256, 256])
        x = self.b1(x) #; print (x.shape)  # torch.Size([2, 24, 256, 256])
        x = self.b2(x) #; print (x.shape)  # torch.Size([2, 32, 128, 128])
        x = self.b3(x) #; print (x.shape)  # torch.Size([2, 48, 64, 64])
        x = self.b4(x) #; print (x.shape)  # torch.Size([2, 96, 32, 32])
        x = self.b5(x) #; print (x.shape)  # torch.Size([2, 136, 32, 32])
        #------------
        mask = self.mask(x)
        #-------------
        x = self.b6(x) #; print (x.shape)  # torch.Size([2, 232, 16, 16])
        x = self.b7(x) #; print (x.shape)  # torch.Size([2, 384, 16, 16])
        x = self.b8(x) #; print (x.shape)  # torch.Size([2, 1536, 16, 16])
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        #x = F.dropout(x, 0.5, training=self.training)
        logit = self.logit(x)
        return logit, mask
    
class NetL(nn.Module):
    def __init__(self):
        super(NetL, self).__init__()

        e = tf_efficientnetv2_l(pretrained=False)
        self.b0 = nn.Sequential(
            e.conv_stem,
            e.bn1,
            e.act1,
        )
        self.b1 = e.blocks[0]
        self.b2 = e.blocks[1]
        self.b3 = e.blocks[2]
        self.b4 = e.blocks[3]
        self.b5 = e.blocks[4]
        self.b6 = e.blocks[5]
        self.b7 = e.blocks[6]
        self.b8 = nn.Sequential(
            e.conv_head, #384, 1536
            e.bn2,
            e.act2,
        )

        self.logit = nn.Linear(1280,4)
        self.mask = nn.Sequential(
            nn.Conv2d(224, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    # @torch.cuda.amp.autocast()
    def forward(self, image):
        batch_size = len(image)
        x = 2*image-1     # ; print('input ',   x.shape)

        x = self.b0(x) #; print (x.shape)  # torch.Size([2, 40, 256, 256])
        x = self.b1(x) #; print (x.shape)  # torch.Size([2, 24, 256, 256])
        x = self.b2(x) #; print (x.shape)  # torch.Size([2, 32, 128, 128])
        x = self.b3(x) #; print (x.shape)  # torch.Size([2, 48, 64, 64])
        x = self.b4(x) #; print (x.shape)  # torch.Size([2, 96, 32, 32])
        x = self.b5(x) #; print (x.shape)  # torch.Size([2, 136, 32, 32])
        #------------
        mask = self.mask(x)
        #-------------
        x = self.b6(x) #; print (x.shape)  # torch.Size([2, 232, 16, 16])
        x = self.b7(x) #; print (x.shape)  # torch.Size([2, 384, 16, 16])
        x = self.b8(x) #; print (x.shape)  # torch.Size([2, 1536, 16, 16])
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        #x = F.dropout(x, 0.5, training=self.training)
        logit = self.logit(x)
        return logit, mask

# Torch Get Pretrained Net

In [ ]:
study_torch_384size = 384
study_torch_480size = 480
study_torch_440size = 440
study_torch_540size = 540
study_torch_640size = 640
study_torch_size = 640
study_tf_size = 512
image_size = 640
twoclass_size = 512
twoclass_480size = 480
twoclass_384size = 384

effv_model_path_384 = "../input/onecycle-effv2-384/"
effv_model_path_480 = "../input/480imageeffv2cv376/"
effv_large_model_path_440 = "../input/onecycle-effv2-large-440/"
effv_large_model_path_540 = "../input/onecycle-effv2-large-540/"
effv_large_model_path_640 = "../input/effvlarge640/"
effv_model_twoclass_384 = "../input/efftwoclass384/"
effv_model_twoclass_480 = "../input/efftwoclass480/"
torch_model_384list = os.listdir(effv_model_path_384)
torch_model_480list = os.listdir(effv_model_path_480)
torch_model_large_440list = os.listdir(effv_large_model_path_440)
torch_model_large_540list = os.listdir(effv_large_model_path_540)
torch_model_large_640list = os.listdir(effv_large_model_path_640)
def get_pretrained_model(path, model_name='m'):
    checkpoint = path
    if model_name == 'm':
        net = Net()
    elif model_name == 'l':
        net = NetL()
    if torch.cuda.is_available():
        net.load_state_dict(torch.load(path)['state_dict'], strict=True)
        net.cuda()
    else:
        net.load_state_dict(torch.load(path, map_location=torch.device('cpu'))['state_dict'], strict=True)
    return net

pretrained_384path = [effv_model_path_384 + x for x in torch_model_384list]
pretrained_480path = [effv_model_path_480 + x for x in torch_model_480list]
pretrained_large_440path = [effv_large_model_path_440 + x for x in torch_model_large_440list]
pretrained_large_540path = [effv_large_model_path_540 + x for x in torch_model_large_540list]
pretrained_440path = [effv_large_model_path_440 + x for x in torch_model_large_440list]
pretrained_540path = [effv_large_model_path_540 + x for x in torch_model_large_540list]
pretrained_large_640path = [effv_large_model_path_640 + x for x in torch_model_large_640list]
pretrained_640path = [effv_large_model_path_640 + x for x in torch_model_large_640list]

# Torch Study Prediction

In [ ]:
def do_predict_384(net, valid_loader, tta=['flip','scale']): #flip
    
    valid_probability = []
    

    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        
        if torch.cuda.is_available():
            image  = batch['image'].cuda()
        else:
            image  = batch['image']

        #<todo> TTA
        net.eval()
        with torch.no_grad():
            probability = []
            logit, mask = net(image)
            probability.append(torch.sigmoid(logit))

            if 'flip' in tta:
                logit, mask = net(torch.flip(image,dims=(3,)))
                probability.append(torch.sigmoid(logit))

            if 'scale' in tta:
                # size=None, scale_factor=None, mode='nearest', align_corners=None, recompute_scale_factor=None):
                logit, mask = net(F.interpolate(image, scale_factor=1.25, mode='bilinear', align_corners=False))
                probability.append(torch.sigmoid(logit))

            #--------------
            probability = torch.stack(probability,0).mean(0)

        
        valid_probability.append(probability.data.cpu().numpy())

    probability = np.concatenate(valid_probability)
    return probability

def do_predict_480(net, valid_loader, tta=['flip','scale']): #flip
    
    valid_probability = []
    

    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        
        if torch.cuda.is_available():
            image  = batch['image'].cuda()
        else:
            image  = batch['image']

        #<todo> TTA
        net.eval()
        with torch.no_grad():
            probability = []
            logit, mask = net(image)
            probability.append(torch.sigmoid(logit))

            if 'flip' in tta:
                logit, mask = net(torch.flip(image,dims=(3,)))
                probability.append(torch.sigmoid(logit))

            if 'scale' in tta:
                # size=None, scale_factor=None, mode='nearest', align_corners=None, recompute_scale_factor=None):
                size = random.uniform(1,1.1)
                logit, mask = net(F.interpolate(image, scale_factor=size, mode='bilinear', align_corners=False))
                probability.append(torch.sigmoid(logit))

            #--------------
            probability = torch.stack(probability,0).mean(0)

        
        valid_probability.append(probability.data.cpu().numpy())

    probability = np.concatenate(valid_probability)
    return probability

def do_predict_440(net, valid_loader, tta=['flip','scale']): #flip
    
    valid_probability = []
    

    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        
        if torch.cuda.is_available():
            image  = batch['image'].cuda()
        else:
            image  = batch['image']

        #<todo> TTA
        net.eval()
        with torch.no_grad():
            probability = []
            logit, mask = net(image)
            probability.append(torch.sigmoid(logit))

            if 'flip' in tta:
                logit, mask = net(torch.flip(image,dims=(3,)))
                probability.append(torch.sigmoid(logit))

            if 'scale' in tta:
                # size=None, scale_factor=None, mode='nearest', align_corners=None, recompute_scale_factor=None):
                size = random.uniform(1,1.15)
                logit, mask = net(F.interpolate(image, scale_factor=size, mode='bilinear', align_corners=False))
                probability.append(torch.sigmoid(logit))

            #--------------
            probability = torch.stack(probability,0).mean(0)

        
        valid_probability.append(probability.data.cpu().numpy())

    probability = np.concatenate(valid_probability)
    return probability


def do_predict_540(net, valid_loader, tta=['flip','scale']): #flip
    
    valid_probability = []
    

    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        
        if torch.cuda.is_available():
            image  = batch['image'].cuda()
        else:
            image  = batch['image']

        #<todo> TTA
        net.eval()
        with torch.no_grad():
            probability = []
            logit, mask = net(image)
            probability.append(torch.sigmoid(logit))

            if 'flip' in tta:
                logit, mask = net(torch.flip(image,dims=(3,)))
                probability.append(torch.sigmoid(logit))

            if 'scale' in tta:
                # size=None, scale_factor=None, mode='nearest', align_corners=None, recompute_scale_factor=None):
                size = random.uniform(0.95,1.05)
                logit, mask = net(F.interpolate(image, scale_factor=size, mode='bilinear', align_corners=False))
                probability.append(torch.sigmoid(logit))

            #--------------
            probability = torch.stack(probability,0).mean(0)

        
        valid_probability.append(probability.data.cpu().numpy())

    probability = np.concatenate(valid_probability)
    return probability

def do_predict_640(net, valid_loader, tta=['flip','scale']): #flip
    
    valid_probability = []
    

    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        
        if torch.cuda.is_available():
            image  = batch['image'].cuda()
        else:
            image  = batch['image']

        #<todo> TTA
        net.eval()
        with torch.no_grad():
            probability = []
            logit, mask = net(image)
            probability.append(torch.sigmoid(logit))

            if 'flip' in tta:
                logit, mask = net(torch.flip(image,dims=(3,)))
                probability.append(torch.sigmoid(logit))

            if 'scale' in tta:
                # size=None, scale_factor=None, mode='nearest', align_corners=None, recompute_scale_factor=None):
                size = random.uniform(0.85,1.05)
                logit, mask = net(F.interpolate(image, scale_factor=size, mode='bilinear', align_corners=False))
                probability.append(torch.sigmoid(logit))

            #--------------
            probability = torch.stack(probability,0).mean(0)

        
        valid_probability.append(probability.data.cpu().numpy())

    probability = np.concatenate(valid_probability)
    return probability

In [ ]:
import gc
model0 = pretrained_384path[0]
predict0 = do_predict_384(get_pretrained_model(model0, 'm'), test_384loader, tta=['flip','scale'])
del model0
gc.collect()
model1 = pretrained_384path[1]
predict1 = do_predict_384(get_pretrained_model(model1, 'm'), test_384loader, tta=['flip','scale'])
del model1
gc.collect()
model2 = pretrained_384path[2]
predict2 = do_predict_384(get_pretrained_model(model2, 'm'), test_384loader, tta=['flip','scale'])
del model2
gc.collect()
model3 = pretrained_384path[3]
predict3 = do_predict_384(get_pretrained_model(model3, 'm'), test_384loader, tta=['flip','scale'])
del model3
gc.collect()
model4 = pretrained_384path[4]
predict4 = do_predict_384(get_pretrained_model(model4, 'm'), test_384loader, tta=['flip','scale'])
gc.collect()
del model4

In [ ]:
torch_384value = ((predict0) + (predict1) + (predict2) + (predict3) + (predict4))*(0.2)
#sub_df[label_cols] = 

#del model0, model1, model2, model3, model4
import gc
gc.collect()

In [ ]:
#import gc
#model0 = pretrained_480path[0]
#predict0 = do_predict_480(get_pretrained_model(model0, 'm'), test_480loader, tta=['flip','scale'])
#del model0
#gc.collect()
#model1 = pretrained_480path[1]
#predict1 = do_predict_480(get_pretrained_model(model1, 'm'), test_480loader, tta=['flip','scale'])
#del model1
#gc.collect()
#model2 = pretrained_480path[2]
#predict2 = do_predict_480(get_pretrained_model(model2, 'm'), test_480loader, tta=['flip','scale'])
#del model2
#gc.collect()
#model3 = pretrained_480path[3]
#predict3 = do_predict_480(get_pretrained_model(model3, 'm'), test_480loader, tta=['flip','scale'])
#del model3
#gc.collect()
#model4 = pretrained_480path[4]
#predict4 = do_predict_480(get_pretrained_model(model4, 'm'), test_480loader, tta=['flip','scale'])
#gc.collect()
#del model4

In [ ]:
#torch_480value = ((predict0) * (predict1)*(predict2) * (predict3) * (predict4))**(0.2)
#sub_df[label_cols] = 

#del model0, model1, model2, model3, model4
#import gc
#gc.collect()

In [ ]:
import gc
model0 = pretrained_440path[0]
predict0 = do_predict_440(get_pretrained_model(model0, 'l'), test_440loader, tta=['flip','scale'])
del model0
gc.collect()
model1 = pretrained_440path[1]
predict1 = do_predict_440(get_pretrained_model(model1, 'l'), test_440loader, tta=['flip','scale'])
del model1
gc.collect()
model2 = pretrained_440path[2]
predict2 = do_predict_440(get_pretrained_model(model2, 'l'), test_440loader, tta=['flip','scale'])
del model2
gc.collect()
model3 = pretrained_440path[3]
predict3 = do_predict_440(get_pretrained_model(model3, 'l'), test_440loader, tta=['flip','scale'])
del model3
gc.collect()
model4 = pretrained_440path[4]
predict4 = do_predict_440(get_pretrained_model(model4, 'l'), test_440loader, tta=['flip','scale'])
gc.collect()
del model4

In [ ]:
#torch_440value = np.stack([predict0**0.25, predict1**0.25, predict2**0.25, predict3**0.25, predict4**0.25]).mean(axis=0)
torch_440value = ((predict0) + (predict1) + (predict2) + (predict3) + (predict4))*(0.2)
#sub_df[label_cols] = 

#del model0, model1, model2, model3, model4
import gc
gc.collect()

In [ ]:
import gc
model0 = pretrained_540path[0]
predict0 = do_predict_540(get_pretrained_model(model0, 'l'), test_540loader, tta=['flip','scale'])
del model0
gc.collect()
model1 = pretrained_540path[1]
predict1 = do_predict_540(get_pretrained_model(model1, 'l'), test_540loader, tta=['flip','scale'])
del model1
gc.collect()
model2 = pretrained_540path[2]
predict2 = do_predict_540(get_pretrained_model(model2, 'l'), test_540loader, tta=['flip','scale'])
del model2
gc.collect()
model3 = pretrained_540path[3]
predict3 = do_predict_540(get_pretrained_model(model3, 'l'), test_540loader, tta=['flip','scale'])
del model3
gc.collect()
model4 = pretrained_540path[4]
predict4 = do_predict_540(get_pretrained_model(model4, 'l'), test_540loader, tta=['flip','scale'])
gc.collect()
del model4

In [ ]:
torch_540value = ((predict0) + (predict1) + (predict2) + (predict3) + (predict4))*(0.2)
#sub_df[label_cols] = 

#del model0, model1, model2, model3, model4
import gc
gc.collect()

In [ ]:
import tensorflow as tf
import tensorflow_hub as tfhub

MODEL_ARCH = 'efficientnetv2-l-21k-ft1k'
# Get the TensorFlow Hub model URL
hub_type = 'feature_vector' # ['classification', 'feature_vector']
MODEL_ARCH_PATH = f'/kaggle/input/efficientnetv2-tfhub-weight-files/tfhub_models/{MODEL_ARCH}/{hub_type}'

# Custom wrapper class to load the right pretrained weights explicitly from the local directory
class KerasLayerWrapper(tfhub.KerasLayer):
    def __init__(self, handle, **kwargs):
        handle = tfhub.KerasLayer(tfhub.load(MODEL_ARCH_PATH))
        super().__init__(handle, **kwargs)

In [ ]:
#!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

import os

import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
import tensorflow as tf
import random

MODEL_PATH = '/kaggle/input/pesudo-512512'
#test_paths = study_df.image_path.tolist()
BATCH_SIZE = 32

def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img

    def decode_with_labels(path, label):
        return decode(path), label

    return decode_with_labels if with_labels else decode

def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        #img = tf.image.random_flip_up_down(img)
        img = tf.image.random_brightness(img, 0.1)
        return img

    def augment_with_labels(img, label):
        return augment(img), label

    return augment_with_labels if with_labels else augment

def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)

    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)

    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)

    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)

    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)

    return dset

# strategy = auto_select_accelerator()
# BATCH_SIZE = strategy.num_replicas_in_sync * 16

#label_cols = ['negative', 'typical', 'indeterminate', 'atypical']
#study_df[label_cols] = 0
test_paths = f'/kaggle/tmp/{split}/study_torch_size/' + sub_df['id'] +'.png'
test_decoder = build_decoder(with_labels=False,
                             target_size=(512,
                                          512), ext='png')


with tf.device('/device:GPU:0'):
    models = []
    models0 = tf.keras.models.load_model(f'{MODEL_PATH}/model0.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models1 = tf.keras.models.load_model(f'{MODEL_PATH}/model1.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models2 = tf.keras.models.load_model(f'{MODEL_PATH}/model2.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models3 = tf.keras.models.load_model(f'{MODEL_PATH}/model3.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models4 = tf.keras.models.load_model(f'{MODEL_PATH}/model4.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models.append(models0)
    models.append(models1)
    models.append(models2)
    models.append(models3)
    models.append(models4)

#study_df[label_cols] = sum([model.predict(test_dataset, verbose=1) for model in models]) / len(models)
#study_df['PredictionString'] = study_df[label_cols].apply(lambda row: f'negative {row.negative} 0 0 1 1 typical {row.typical} 0 0 1 1 indeterminate {row.indeterminate} 0 0 1 1 atypical {row.atypical} 0 0 1 1', axis=1)

test_dataset = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)
tensor_value_list1= [model.predict(test_dataset, verbose=1) for model in models]
test_dataset = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=True, cache=False,
    decode_fn=test_decoder
)
tensor_value_list2= [model.predict(test_dataset, verbose=1) for model in models]

tensor_value_512 = (tensor_value_list1[0]+tensor_value_list1[1]+tensor_value_list1[2]+tensor_value_list1[3]+tensor_value_list1[4]+tensor_value_list2[0]+tensor_value_list2[1]+tensor_value_list2[2]+tensor_value_list2[3]+tensor_value_list2[4])*(0.1)
del models0, models1, models2, models3, models4
del models
del test_dataset, test_decoder
del tensor_value_list1
del tensor_value_list2
import gc
gc.collect()


##################################
########### 640 SIZE #############
##################################
MODEL_PATH = '/kaggle/input/simm-640x640'
test_paths = f'/kaggle/tmp/{split}/study_torch_size/' + sub_df['id'] +'.png'
test_decoder = build_decoder(with_labels=False,
                             target_size=(640,
                                          640), ext='png')


with tf.device('/device:GPU:0'):
    models = []
    models0 = tf.keras.models.load_model(f'{MODEL_PATH}/model0.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models1 = tf.keras.models.load_model(f'{MODEL_PATH}/model1.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models2 = tf.keras.models.load_model(f'{MODEL_PATH}/model2.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models3 = tf.keras.models.load_model(f'{MODEL_PATH}/model3.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models4 = tf.keras.models.load_model(f'{MODEL_PATH}/model4.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models.append(models0)
    models.append(models1)
    models.append(models2)
    models.append(models3)
    models.append(models4)

#study_df[label_cols] = sum([model.predict(test_dataset, verbose=1) for model in models]) / len(models)
#study_df['PredictionString'] = study_df[label_cols].apply(lambda row: f'negative {row.negative} 0 0 1 1 typical {row.typical} 0 0 1 1 indeterminate {row.indeterminate} 0 0 1 1 atypical {row.atypical} 0 0 1 1', axis=1)
test_dataset = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)
tensor_value_list1= [model.predict(test_dataset, verbose=1) for model in models]
test_dataset = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=True, cache=False,
    decode_fn=test_decoder
)
tensor_value_list2= [model.predict(test_dataset, verbose=1) for model in models]

tensor_value_640 = (tensor_value_list1[0]+tensor_value_list1[1]+tensor_value_list1[2]+tensor_value_list1[3]+tensor_value_list1[4]+tensor_value_list2[0]+tensor_value_list2[1]+tensor_value_list2[2]+tensor_value_list2[3]+tensor_value_list2[4])*(0.1)
del models0, models1, models2, models3, models4
del models
del test_dataset, test_decoder
del tensor_value_list1
del tensor_value_list2
import gc
gc.collect()

In [ ]:
import tensorflow as tf
import tensorflow_hub as tfhub

MODEL_ARCH = 'efficientnetv2-m-21k-ft1k'
# Get the TensorFlow Hub model URL
hub_type = 'feature_vector' # ['classification', 'feature_vector']
MODEL_ARCH_PATH = f'/kaggle/input/efficientnetv2-tfhub-weight-files/tfhub_models/{MODEL_ARCH}/{hub_type}'

# Custom wrapper class to load the right pretrained weights explicitly from the local directory
class KerasLayerWrapper(tfhub.KerasLayer):
    def __init__(self, handle, **kwargs):
        handle = tfhub.KerasLayer(tfhub.load(MODEL_ARCH_PATH))
        super().__init__(handle, **kwargs)

##################################
########### 480 SIZE #############
##################################        

MODEL_PATH = '/kaggle/input/covid-480x480-2st'
test_paths = f'/kaggle/tmp/{split}/study_torch_size/' + sub_df['id'] +'.png'
test_decoder = build_decoder(with_labels=False,
                             target_size=(480,
                                          480), ext='png')


with tf.device('/device:GPU:0'):
    models = []
    models0 = tf.keras.models.load_model(f'{MODEL_PATH}/model0.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models1 = tf.keras.models.load_model(f'{MODEL_PATH}/model1.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models2 = tf.keras.models.load_model(f'{MODEL_PATH}/model2.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models3 = tf.keras.models.load_model(f'{MODEL_PATH}/model3.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models4 = tf.keras.models.load_model(f'{MODEL_PATH}/model4.h5',
                                         custom_objects={'KerasLayer': KerasLayerWrapper})
    models.append(models0)
    models.append(models1)
    models.append(models2)
    models.append(models3)
    models.append(models4)

#study_df[label_cols] = sum([model.predict(test_dataset, verbose=1) for model in models]) / len(models)
#study_df['PredictionString'] = study_df[label_cols].apply(lambda row: f'negative {row.negative} 0 0 1 1 typical {row.typical} 0 0 1 1 indeterminate {row.indeterminate} 0 0 1 1 atypical {row.atypical} 0 0 1 1', axis=1)
test_dataset = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)
tensor_value_list1= [model.predict(test_dataset, verbose=1) for model in models]
test_dataset = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=True, cache=False,
    decode_fn=test_decoder
)
tensor_value_list2= [model.predict(test_dataset, verbose=1) for model in models]


tensor_value_480 = (tensor_value_list1[0]+tensor_value_list1[1]+tensor_value_list1[2]+tensor_value_list1[3]+tensor_value_list1[4]+tensor_value_list2[0]+tensor_value_list2[1]+tensor_value_list2[2]+tensor_value_list2[3]+tensor_value_list2[4])*(0.1)
del models0, models1, models2, models3, models4
del models
del test_dataset, test_decoder
del tensor_value_list1
del tensor_value_list2
import gc
gc.collect()

In [ ]:
#!pip install /kaggle/input/kerasapplications -q
#!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

import os

import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
import tensorflow as tf
import random

def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy


def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img

    def decode_with_labels(path, label):
        return decode(path), label

    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        
        random_number = random.randint(0,2)
        if random_number == 1:
            img = tf.image.adjust_brightness(img, 1.2)
        if random_number == 2:
            img = tf.image.adjust_brightness(img, 0.8)
        return img

    def augment_with_labels(img, label):
        return augment(img), label

    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)

    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)

    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)

    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)

    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)

    return dset

#COMPETITION_NAME = "siim-cov19-test-img512-study-600"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 8

IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 512)

#load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
#if fast_sub:
#    sub_df = fast_df.copy()
#else:
#    sub_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
#sub_df = sub_df[:study_len]
#test_paths = f'/kaggle/tmp/{split}/study_tf/' + sub_df['id'] +'.png'

#sub_df['negative'] = 0
#sub_df['typical'] = 0
#sub_df['indeterminate'] = 0
#sub_df['atypical'] = 0


#label_cols = sub_df.columns[2:]

#test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[7], IMSIZE[7]), ext='png')
#dtest = build_dataset(
#    test_paths, bsize=BATCH_SIZE, repeat=False, 
#    shuffle=False, augment=False, cache=False,
#    decode_fn=test_decoder
#)

#with strategy.scope():
    
#    models = []
#    
#    models0 = tf.keras.models.load_model(
#        '../input/siim-covid19-efnb7-train-study/model0.h5'
#    )
#    models1 = tf.keras.models.load_model(
#        '../input/siim-covid19-efnb7-train-study/model1.h5'
#    )
#    models2 = tf.keras.models.load_model(
#        '../input/siim-covid19-efnb7-train-study/model2.h5'
#    )
#    models3 = tf.keras.models.load_model(
#        '../input/siim-covid19-efnb7-train-study/model3.h5'
#    )
#    models4 = tf.keras.models.load_model(
#        '../input/siim-covid19-efnb7-train-study/model4.h5'
#    )
    
#    models.append(models0)
#    models.append(models1)
#    models.append(models2)
#    models.append(models3)
#    models.append(models4)
    

#tensor_value_list = [model.predict(dtest, verbose=1) for model in models]
#tensor_value = (tensor_value_list[0]*tensor_value_list[1]*tensor_value_list[2]*tensor_value_list[3]*tensor_value_list[4])**(0.2)
#del models0, models1, models2, models3, models4
#del models
#import gc
#gc.collect()

In [ ]:
sub_df[label_cols] = (torch_384value + torch_440value + tensor_value_480 + tensor_value_640 + torch_540value + tensor_value_512) * (1/6)

In [ ]:
sub_df.columns = ['id', 'PredictionString1', 'negative', 'typical', 'indeterminate', 'atypical']
df = pd.merge(df, sub_df, on = 'id', how = 'left')

# study string

In [ ]:
for i in range(study_len):
    negative = df.loc[i,'negative']
    typical = df.loc[i,'typical']
    indeterminate = df.loc[i,'indeterminate']
    atypical = df.loc[i,'atypical']
    df.loc[i, 'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'

In [ ]:
df_study = df[['id', 'PredictionString']]

# 2 Class Prediction

# DataLoader Define

In [ ]:
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
sub_df = sub_df[study_len:]
test_paths = f'/kaggle/tmp/{split}/image/' + sub_df['id'] +'.png'
sub_df['none'] = 0
label_cols = sub_df.columns[2]

In [ ]:
def null_augment(r):
    image = r['image']
    # if image[:2].shape != (image_size, image_size):
    #     r['image'] = cv2.resize(image, dsize=(image_size, image_size), interpolation=cv2.INTER_AREA)
    return r


class TwoClassSiimDataset(Dataset):
    def __init__(self, df, image_size, augment=null_augment):
        super().__init__()
        self.df = df
        self.augment = augment
        self.length = len(df)
        self.image_size = image_size

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        d = self.df.iloc[index]

        #image_file = data_dir + '/%s_640/%s/%s/%s.png' % (d.set, d.study, d.series, d.image)
        #image_file = data_dir + '/%s_full_512/%s.png' % (d.set, d.image)
        image_file = f'/kaggle/tmp/{split}/image/' + d['id'].split("_")[0] + "_image.png"
        #print(image_file)
        image = cv2.imread(image_file,cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.image_size, self.image_size))

        r = {'image' : image, 'image_size' : self.image_size}
        if self.augment is not None: r = self.augment(r)
        return r
    
def twoclass_null_collate(batch):
    collate = defaultdict(list)

    for r in batch:
        for k, v in r.items():
            collate[k].append(v)

    # ---
    batch_size = len(batch)
    image = np.stack(collate['image'])
    image = image.reshape(batch_size, 1, collate['image_size'][0],collate['image_size'][0]).repeat(3,1)
    image = np.ascontiguousarray(image)
    image = image.astype(np.float32) / 255
    collate['image'] = torch.from_numpy(image)

    return collate



In [ ]:
twoclass_384dataset = TwoClassSiimDataset(sub_df, 384)
twoclass_384loader  = DataLoader(
                twoclass_384dataset,
                sampler = SequentialSampler(twoclass_384dataset),
                batch_size  = 16,#128, #
                drop_last   = False,
                num_workers = 0,
                pin_memory  = True,
                collate_fn  = twoclass_null_collate,
            )

twoclass_480dataset = TwoClassSiimDataset(sub_df, 480)
twoclass_480loader  = DataLoader(
                twoclass_480dataset,
                sampler = SequentialSampler(twoclass_480dataset),
                batch_size  = 16,#128, #
                drop_last   = False,
                num_workers = 0,
                pin_memory  = True,
                collate_fn  = twoclass_null_collate,
            )

# Get Two Class Pretrained model

In [ ]:
class TwoClassNet(nn.Module):
    def __init__(self):
        super(TwoClassNet, self).__init__()

        e = tf_efficientnetv2_m(pretrained=False)
        self.b0 = nn.Sequential(
            e.conv_stem,
            e.bn1,
            e.act1,
        )
        self.b1 = e.blocks[0]
        self.b2 = e.blocks[1]
        self.b3 = e.blocks[2]
        self.b4 = e.blocks[3]
        self.b5 = e.blocks[4]
        self.b6 = e.blocks[5]
        self.b7 = e.blocks[6]
        self.b8 = nn.Sequential(
            e.conv_head, #384, 1536
            e.bn2,
            e.act2,
        )

        self.logit = nn.Linear(1280,2)
        self.mask = nn.Sequential(
            nn.Conv2d(176, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )


    # @torch.cuda.amp.autocast()
    def forward(self, image):
        batch_size = len(image)
        x = 2*image-1     # ; print('input ',   x.shape)

        x = self.b0(x) #; print (x.shape)  # torch.Size([2, 40, 256, 256])
        x = self.b1(x) #; print (x.shape)  # torch.Size([2, 24, 256, 256])
        x = self.b2(x) #; print (x.shape)  # torch.Size([2, 32, 128, 128])
        x = self.b3(x) #; print (x.shape)  # torch.Size([2, 48, 64, 64])
        x = self.b4(x) #; print (x.shape)  # torch.Size([2, 96, 32, 32])
        x = self.b5(x) #; print (x.shape)  # torch.Size([2, 136, 32, 32])
        #------------
        mask = self.mask(x)
        #-------------
        x = self.b6(x) #; print (x.shape)  # torch.Size([2, 232, 16, 16])
        x = self.b7(x) #; print (x.shape)  # torch.Size([2, 384, 16, 16])
        x = self.b8(x) #; print (x.shape)  # torch.Size([2, 1536, 16, 16])
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size,-1)
        #x = F.dropout(x, 0.5, training=self.training)
        logit = self.logit(x)
        return logit, mask

In [ ]:
#torch_model_list = os.listdir("../input/effv2-cv375/")
effv_model_twoclass_384 = "../input/efftwoclass384/"
effv_model_twoclass_480 = "../input/efftwoclass480/"
torch_twoclass_384 = os.listdir(effv_model_twoclass_384)
torch_twoclass_480 = os.listdir(effv_model_twoclass_480)
def twoclass_get_pretrained_model(path):
    checkpoint = path
    net = TwoClassNet()
    if torch.cuda.is_available():
        net.load_state_dict(torch.load(path)['state_dict'], strict=True)
        net.cuda()
    else:
        net.load_state_dict(torch.load(path, map_location=torch.device('cpu'))['state_dict'], strict=True)
    return net

twoclass_384 = [effv_model_twoclass_384 + x for x in torch_twoclass_384]
twoclass_480 = [effv_model_twoclass_480 + x for x in torch_twoclass_480]

# Two Class Prediction define

In [ ]:
def twoclass_predict_384(net, valid_loader, tta=['flip','scale']): #flip
    
    valid_probability = []
    

    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        
        if torch.cuda.is_available():
            image  = batch['image'].cuda()
        else:
            image  = batch['image']

        #<todo> TTA
        net.eval()
        with torch.no_grad():
            probability = []
            logit, mask = net(image)
            probability.append(torch.sigmoid(logit))

            if 'flip' in tta:
                logit, mask = net(torch.flip(image,dims=(3,)))
                probability.append(torch.sigmoid(logit))

            if 'scale' in tta:
                # size=None, scale_factor=None, mode='nearest', align_corners=None, recompute_scale_factor=None):
                logit, mask = net(F.interpolate(image, scale_factor=1.25, mode='bilinear', align_corners=False))
                probability.append(torch.sigmoid(logit))

            #--------------
            probability = torch.stack(probability,0).mean(0)

        
        valid_probability.append(probability.data.cpu().numpy())

    probability = np.concatenate(valid_probability)
    return probability

def twoclass_predict_480(net, valid_loader, tta=['flip','scale']): #flip
    
    valid_probability = []
    

    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        
        if torch.cuda.is_available():
            image  = batch['image'].cuda()
        else:
            image  = batch['image']

        #<todo> TTA
        net.eval()
        with torch.no_grad():
            probability = []
            logit, mask = net(image)
            probability.append(torch.sigmoid(logit))

            if 'flip' in tta:
                logit, mask = net(torch.flip(image,dims=(3,)))
                probability.append(torch.sigmoid(logit))

            if 'scale' in tta:
                # size=None, scale_factor=None, mode='nearest', align_corners=None, recompute_scale_factor=None):
                size = random.uniform(1,1.1)
                logit, mask = net(F.interpolate(image, scale_factor=size, mode='bilinear', align_corners=False))
                probability.append(torch.sigmoid(logit))

            #--------------
            probability = torch.stack(probability,0).mean(0)

        
        valid_probability.append(probability.data.cpu().numpy())

    probability = np.concatenate(valid_probability)
    return probability

# Two class Prediction

In [ ]:
import gc
model0 = twoclass_384[0]
predict0 = twoclass_predict_384(twoclass_get_pretrained_model(model0), twoclass_384loader, tta=['flip','scale'])
del model0
gc.collect()
import gc
model1 = twoclass_384[1]
predict1 = twoclass_predict_384(twoclass_get_pretrained_model(model1), twoclass_384loader, tta=['flip','scale'])
del model1
gc.collect()
import gc
model2 = twoclass_384[2]
predict2 = twoclass_predict_384(twoclass_get_pretrained_model(model2), twoclass_384loader, tta=['flip','scale'])
del model2
gc.collect()
import gc
model3 = twoclass_384[3]
predict3 = twoclass_predict_384(twoclass_get_pretrained_model(model3), twoclass_384loader, tta=['flip','scale'])
del model3
gc.collect()
import gc
model4 = twoclass_384[4]
predict4 = twoclass_predict_384(twoclass_get_pretrained_model(model4), twoclass_384loader, tta=['flip','scale'])
del model4
gc.collect()

In [ ]:
twoclass_384value = np.stack([predict0, predict1, predict2, predict3, predict4]).mean(axis=0)
#sub_df[label_cols] = 

import gc
gc.collect()

In [ ]:
import gc
model0 = twoclass_480[0]
predict0 = twoclass_predict_480(twoclass_get_pretrained_model(model0), twoclass_480loader, tta=['flip','scale'])
del model0
gc.collect()
import gc
model1 = twoclass_480[1]
predict1 = twoclass_predict_480(twoclass_get_pretrained_model(model1), twoclass_480loader, tta=['flip','scale'])
del model1
gc.collect()
import gc
model2 = twoclass_480[2]
predict2 = twoclass_predict_480(twoclass_get_pretrained_model(model2), twoclass_480loader, tta=['flip','scale'])
del model2
gc.collect()
import gc
model3 = twoclass_480[3]
predict3 = twoclass_predict_480(twoclass_get_pretrained_model(model3), twoclass_480loader, tta=['flip','scale'])
del model3
gc.collect()
import gc
model4 = twoclass_480[4]
predict4 = twoclass_predict_480(twoclass_get_pretrained_model(model4), twoclass_480loader, tta=['flip','scale'])
del model4
gc.collect()

In [ ]:
twoclass_480value = np.stack([predict0, predict1, predict2, predict3, predict4]).mean(axis=0)
#sub_df[label_cols] = 

import gc
gc.collect()

# TensorFLow TwoClass

In [ ]:
test_decoder = build_decoder(with_labels=False, target_size=(512, 512), ext='png')
dtest = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)

with strategy.scope():
    
    models = []
    
    models0 = tf.keras.models.load_model(
        '/kaggle/input/siim-covid19-efnb7-train-fold0-5-2class/model0.h5'
    )
    models1 = tf.keras.models.load_model(
        '/kaggle/input/siim-covid19-efnb7-train-fold0-5-2class/model1.h5'
    )
    models2 = tf.keras.models.load_model(
        '/kaggle/input/siim-covid19-efnb7-train-fold0-5-2class/model2.h5'
    )
    models3 = tf.keras.models.load_model(
        '/kaggle/input/siim-covid19-efnb7-train-fold0-5-2class/model3.h5'
    )
    models4 = tf.keras.models.load_model(
        '/kaggle/input/siim-covid19-efnb7-train-fold0-5-2class/model4.h5'
    )
    
    models.append(models0)
    models.append(models1)
    models.append(models2)
    models.append(models3)
    models.append(models4)
    
tensor_value = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)

In [ ]:
sub_df[label_cols] = ((twoclass_384value + twoclass_480value)[:,0] + tensor_value)/3

In [ ]:
sub_df.to_csv("/kaggle/working/df2class.csv", index=False)

In [ ]:
del models0, models1, models2, models3, models4
del models
gc.collect()

In [ ]:
#sub_df[label_cols] = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)
df_2class = sub_df.reset_index(drop=True)

In [ ]:
import gc
from numba import cuda
import torch

# yolov5 predict

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch

In [ ]:
meta = meta[meta['split'] == 'test']
if fast_sub:
    test_df = fast_df.copy()
else:
    test_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
test_df = df[study_len:].reset_index(drop=True) 
meta['image_id'] = meta['image_id'] + '_image'
meta.columns = ['id', 'dim0', 'dim1', 'split']
test_df = pd.merge(test_df, meta, on = 'id', how = 'left')

# Yolo Prediction

# Load Pytorch Image Loader

In [ ]:
# dim0 -> height
# dim1 -> width
meta['image'] = meta['id'].map(lambda x: x.split("_")[0])

# Make Fold

In [ ]:
study_name_to_predict_string = {
    'Negative for Pneumonia'  :'negative',
    'Typical Appearance'      :'typical',
    'Indeterminate Appearance':'indeterminate',
    'Atypical Appearance'     :'atypical',
}

study_name_to_label = {
    'Negative for Pneumonia'  :0,
    'Typical Appearance'      :1,
    'Indeterminate Appearance':2,
    'Atypical Appearance'     :3,
}
study_label_to_name = { v:k for k,v in study_name_to_label.items()}
num_study_label = len(study_name_to_label)

In [ ]:
df_meta  = meta.copy()

df_test = df_meta[df_meta['split']=='test'].copy()
                   
df_annotate = pd.DataFrame()
df_annotate.loc[:,'image_id'] = df_test.image.values
df_annotate = df_annotate.merge(df_meta[['image','dim0','dim1']],left_on='image_id',right_on='image')
df_annotate.loc[:,'class_id'] = 0
df_annotate.loc[:, 'ncx'] = 0
df_annotate.loc[:, 'ncy'] = 0
df_annotate.loc[:, 'nw'] = 0
df_annotate.loc[:, 'nh'] = 0


for l in study_name_to_label.keys():
    df_test.loc[:,l]=0
df_test = df_test.reset_index(drop=True)

# Siim Dataset

In [ ]:
##########################################
# Imput으로 df_annotate과 df_test가 들어감 #
##########################################

class YoloSiimDataset(Dataset):
    def __init__(self, df_annotate, df, image_size=640, augment=null_augment):
        super().__init__()
        self.gb = df_annotate.groupby('image_id') #9eb725cdb713
        self.df = df
        self.augment = augment
        self.length = len(df)
        self.image_size = image_size

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        d = self.df.iloc[index]
        df = self.gb.get_group(d.image).copy()

        g = self.gb.get_group(d.image)
        annotate = g[['class_id', 'ncx', 'ncy', 'nw', 'nh', ]].values
        annotate = annotate[annotate[:, 0] == 1]

        #image_file = data_dir + '/%s_640/%s/%s/%s.png' % (d.set, d.study, d.series, d.image)
        image_file = f'/kaggle/tmp/{split}/image/' + d['id'].split("_")[0] + "_image.png"
        image  = cv2.imread(image_file,cv2.IMREAD_GRAYSCALE)
        image  = cv2.resize(image, (self.image_size, self.image_size))

        r = {
            'd'  : d,
            'df' : df,
            'annotate' : annotate,
            'image'  : image,
            'image_size' : self.image_size
        }
        if self.augment is not None: r = self.augment(r)
        return r

In [ ]:
def yolo_null_collate(batch):
    collate = defaultdict(list)
    for b,r in enumerate(batch):
        for k, v in r.items():
            if k=='annotate':
                collate[k].append(np.hstack(
                    [np.full((len(v),1),b),v]
                ))
            else:
                collate[k].append(v)

    # ---
    batch_size = len(batch)

    image = np.stack(collate['image'])
    image = image.reshape(batch_size, 1, collate['image_size'][0],collate['image_size'][0]).repeat(3,1)
    image = np.ascontiguousarray(image)
    image = image.astype(np.float32) / 255
    collate['image'] = torch.from_numpy(image)

    annotate = np.concatenate(collate['annotate'])
    collate['annotate'] = torch.from_numpy(annotate).float()
    return collate

In [ ]:
len_test = df_test.shape[0]

In [ ]:
yolo_test_dataset_640 = YoloSiimDataset(df_annotate, df_test, image_size = 640, augment=null_augment)
yolo_test_loader_640  = DataLoader(
                yolo_test_dataset_640,
                sampler = SequentialSampler(yolo_test_dataset_640),
                batch_size  = 8,#128, #
                drop_last   = False,
                num_workers = 0,
                pin_memory  = True,
                collate_fn  = yolo_null_collate,
            )

# Load YoloTorch Network

In [ ]:
import sys

In [ ]:
sys.path.append("/kaggle/input/woongyolov5")
sys.path.append("kaggle/input/woong-yolov5")

In [ ]:
from models.yolo import *
from utils.torch_utils import *
from utils.general import bbox_iou, xywh2xyxy

In [ ]:
pretrained_model = "/kaggle/input/woong-yolov5/yolov5m.pt"
pre_model_cfg_file = "/kaggle/input/woong-yolov5/yolov5m.yaml"

#############################################################
################### Configuration ###########################
#############################################################
import numpy as np


num_class = 1

# yolo net
num_head = 3
image_size = 640
feature_size = [
    80, 40, 20
]
feature_stride = [
    8, 16, 32
]

anchor_size = [
    [[ 10,13], [ 16, 30], [ 33, 23]],  # P3/8
    [[ 30,61], [ 62, 45], [ 59,119]],  # P4/16
    [[116,90], [156,198], [373,326]],  # P5/32
]
num_anchor = 3 #per head

def make_norm_anchor_size():
    norm_anchor_size = np.array(anchor_size)/np.array(feature_stride).reshape(3,1,1)
    norm_anchor_size = norm_anchor_size.tolist()
    return norm_anchor_size


#----------------------------------------------

# yolo loss
anchor_match_ratio_threshold = 4
loss_level_balance = [4.0, 1.0, 0.4]
loss_obj_balance = 100
loss_cls_balance = 1
loss_box_balance = 0.05

#nms
nms_objectness_threshold = 0.001
nms_iou_threshold = 0.5
nms_pre_max_num   = 3000
nms_post_max_num  = 100

class YoloNet(nn.Module):
    def __init__(self):
        super(YoloNet, self).__init__()

        model_cfg_file = pre_model_cfg_file
        pretrain_file  = pretrained_model
        e = Model(
            cfg=model_cfg_file,
            ch=3,
            nc=1,
            anchors=np.array(anchor_size).reshape(num_head,-1).tolist(),
        )
        state_dict = torch.load(pretrain_file, map_location=lambda storage, loc: storage)['model'].float().state_dict()
        state_dict = intersect_dicts(state_dict, e.state_dict(), exclude=['anchor'])  # intersect
        e.load_state_dict(state_dict, strict=False)

        #---
        # remove detect layer
        assert (e.save[-3:] == e.model[-1].f)
        removed = list(e.model.children())[:-1]
        self.backbone = torch.nn.Sequential(*removed)
        self.index = e.save
        #print(self.index)

        #---
        self.head = nn.ModuleList([
            nn.Conv2d(192, num_anchor*6, kernel_size=1),
            nn.Conv2d(384, num_anchor*6, kernel_size=1),
            nn.Conv2d(768, num_anchor*6, kernel_size=1),
        ])

        #---
        #<todo> add image classification head
        # self.index.append( ... add feature layer no to use ...)
        #self.logit=nn.Sequential(
        #    nn.Conv2d(768, 1, kernel_size=1),
        #)


    def forward(self, image):
        batch_size = len(image)
        x = 2*image-1

        # yolov5 backbone ----------------------
        # predict = self.e(x)
        z = []
        for m in self.backbone:
            if m.f != -1:  # if not from previous layer
                if isinstance(m.f, int):
                    x = z[m.f]
                else:
                    x = [x if i == -1 else z[i] for i in m.f]
            x = m(x)  # run
            z.append(x if m.i in self.index else None)  # cache output
        z = [z[i] for i in self.index[-3:]]
        #--------------------------------------
        predict = []
        for n in range(num_head):
            p = self.head[n](z[n])
            batch_size, num_anchor_dim, h, w = p.shape
            dim = num_anchor_dim//num_anchor
            p = p.reshape(batch_size, num_anchor, dim, h, w).permute(0, 1, 3, 4, 2).contiguous()
            predict.append(p)

        return predict

# Load Pretrained-File

In [ ]:
#if (len(os.listdir("/kaggle/input/d/deepkim/woong-yolov5")) != 0):
#    yolo_pretrained_path = ["/kaggle/input/d/deepkim/woong-yolov5/"+x for x in os.listdir("/kaggle/input/d/deepkim/woong-yolov5/") if x.startswith("yolov5_fold")]
yolo_pretrained_path_640 = ["../input/woong-yolov5/"+x for x in os.listdir("../input/woong-yolov5/") if x.endswith("best_model.pth")]
yolo_pretrained_path_512 = ["../input/woong-yolov5/"+x for x in os.listdir("../input/woong-yolov5/") if x.endswith("_512.pth")]
#yolo_pretrained_path_384 = ["../input/woong-yolov5/"+x for x in os.listdir("../input/woong-yolov5/") if x.endswith("_384.pth")]

In [ ]:
def yolo_get_pretrained_model(path):
    yolonet = YoloNet()
    if torch.cuda.is_available():
        yolonet.load_state_dict(torch.load(path)['state_dict'], strict=True)
        yolonet.cuda()
    else:
        yolonet.load_state_dict(torch.load(path, map_location=torch.device('cpu'))['state_dict'], strict=True)
    return yolonet

In [ ]:
torch.cuda.is_available()

In [ ]:
yolonet0_640 = yolo_get_pretrained_model(yolo_pretrained_path_640[0])
yolonet1_640 = yolo_get_pretrained_model(yolo_pretrained_path_640[1])
yolonet2_640 = yolo_get_pretrained_model(yolo_pretrained_path_640[2])
yolonet3_640 = yolo_get_pretrained_model(yolo_pretrained_path_640[3])
yolonet4_640 = yolo_get_pretrained_model(yolo_pretrained_path_640[4])



# Utilitys

In [ ]:
def do_non_max_suppression(
    predict_flat,
    nms_objectness_threshold = nms_objectness_threshold,
    nms_iou_threshold = nms_iou_threshold,
    nms_pre_max_num   = nms_pre_max_num,
    nms_post_max_num  = nms_post_max_num,
):
    batch_size = len(predict_flat)
    detection = []
    for b in range(batch_size):
        p = predict_flat[b]

        i = p[..., 4] > nms_objectness_threshold
        p = p[i]
        num = len(p)
        if num==0:
            det = np.zeros((0,5),np.float32) #None
            detection.append(det)
            continue

        box = xywh2xyxy(p[:, :4])
        score = p[:,4]
        #<todo> x[:, 5:] *= x[:, 4:5]  #conf = obj_conf * cls_conf

        if num > nms_pre_max_num:  # excess boxes
            i = score.argsort(descending=True)[:nms_pre_max_num]  # sort by confidence
            box = box[i]
            score = score[i]

        i = torchvision.ops.nms(box, score, nms_iou_threshold)
        if len(i) > nms_post_max_num:
            i = i[:nms_post_max_num]

        #<todo> merge NMS (boxes merged using weighted mean)  # sort by confidence
        box = box[i]
        score = score[i]
        det = torch.cat([box,score[:,None]],-1)
        det = det.data.cpu().numpy()
        detection.append(det)

    return detection


def pyramid_to_flat(data):
    flat = []
    for n in range(num_head):
        p = data[n]
        batch_size, num_anchor, w, h, dim = p.shape
        flat.append(p.reshape(batch_size, -1, dim))
    flat = torch.cat(flat, 1)
    return flat

def infer_prediction(predict):
    device = predict[0].device

    z = []  # inference output
    for n in range(num_head):
        p = predict[n]
        batch_size, num_anchor, w, h, dim = p.shape

        # inference
        aa = torch.FloatTensor(anchor_size[n]).to(device)
        yy, xx = torch.meshgrid([torch.arange(h), torch.arange(w)])
        grid = torch.stack((xx, yy), 2).reshape((1, 1, h, w, 2)).float().to(device)

        d = p.sigmoid()  #objectivesness and class also use sigmoid ...
        d[..., 0:2] = (d[..., 0:2] * 2 - 0.5 + grid) * feature_stride[n]  # xy
        d[..., 2:4] = (d[..., 2:4] * 2) ** 2 * aa.reshape(1, num_anchor, 1, 1, 2)  # wh

        z.append(d)
    return z

# Do Predict

In [ ]:
# do_predict(net, test_loader)
def do_predict(net, valid_loader, tta=[]): #flip

    valid_detection = []

    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        if torch.cuda.is_available():
            image  = batch['image'].cuda()
        else:
            image = batch['image']

        #<todo> TTA
        net.eval()
        with torch.no_grad():
            predict = net(image)
            predict = infer_prediction(predict)
            predict_flat = pyramid_to_flat(predict)

            detection = do_non_max_suppression(
                    predict_flat,
                    nms_objectness_threshold=0.001,
                    nms_iou_threshold=0.5,
                    nms_pre_max_num=500,
                    nms_post_max_num=25,
            )

            

        valid_detection.extend(detection)


    detection = valid_detection
    return detection

In [ ]:
# do_predict(net, test_loader)
sys.path.append("/kaggle/input/weightedboxesfusion/")
from ensemble_boxes import *
def do_ensemble_predict(net, valid_loader, image_size, tta=[]): #flip

    valid_detection = []

    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        if torch.cuda.is_available():
            image  = batch['image'].cuda()
        else:
            image = batch['image']

        #<todo> TTA
        net.eval()
        with torch.no_grad():
            
            ensemble_detection = []
        
            predict_flip = net(torch.flip(image, dims=(3,)))
            predict_flip = infer_prediction(predict_flip)
            predict_flip_flat = pyramid_to_flat(predict_flip)


            detection_flip = do_non_max_suppression(
                        predict_flip_flat,
                        nms_objectness_threshold=0.001,
                        nms_iou_threshold=0.4,
                        nms_pre_max_num=500,
                        nms_post_max_num=25)
            
            predict = net(image)
            predict = infer_prediction(predict)
            predict_flat = pyramid_to_flat(predict)

            #print(predict_flat)
            #print(predict_flat.shape)
            detection = do_non_max_suppression(
                        predict_flat,
                        nms_objectness_threshold=0.001,
                        nms_iou_threshold=0.4,
                        nms_pre_max_num=500,
                        nms_post_max_num=25)
            
            
            for i, (det0, det1) in enumerate(zip(detection_flip, detection)):
                iou_thr = 0.5
                skip_box_thr = 0.0001
                boxes_list = []
                scores_list = []
                weights = [1,1]
                labels_list = [[1]*det0.shape[0], [1]*det1.shape[0]]

                conf0 = det0[:,4].tolist()
                conf1 = det1[:,4].tolist()
                
                det0 = det0.astype(int)
                det1 = det1.astype(int)
                tmp = det0[:,0].copy()
                det0[:,0] = (image_size) - det0[:,2].astype(int)               
                det0[:,2] = (image_size) - tmp.astype(int)

                det0 = np.where(det0<0,0,det0)
                det1 = np.where(det1<0,0,det1)
                det0 = np.where(det0>image_size,1,det0)
                det1 = np.where(det1>image_size,1,det1)
                box0 = (det0[:,:4]/image_size).tolist()
                box1 = (det1[:,:4]/image_size).tolist()
                
                
                boxes_list.append(box0)
                boxes_list.append(box1)

                scores_list.append(conf0)
                scores_list.append(conf1)

                boxes, scores, labels = weighted_boxes_fusion(boxes_list,
                                                  scores_list,
                                                  labels_list,
                                                  weights=weights,
                                                  iou_thr = iou_thr,
                                                  skip_box_thr = skip_box_thr)
                processed_det0 = np.zeros(shape=(len(boxes), 5))
                processed_det0[:,:4] = np.array(boxes)
                processed_det0[:,4] = np.array(scores)
                ensemble_detection.append(processed_det0)

            

        valid_detection.extend(ensemble_detection)


    detection = valid_detection
    return detection

In [ ]:
detection0_640 = do_ensemble_predict(yolonet0_640, yolo_test_loader_640, image_size=640)
del yolonet0_640
gc.collect()
detection1_640 = do_ensemble_predict(yolonet1_640, yolo_test_loader_640, image_size=640)
del yolonet1_640
gc.collect()

detection2_640 = do_ensemble_predict(yolonet2_640, yolo_test_loader_640, image_size=640)
del yolonet2_640
gc.collect()
detection3_640 = do_ensemble_predict(yolonet3_640, yolo_test_loader_640, image_size=640)
del yolonet3_640
gc.collect()
detection4_640 = do_ensemble_predict(yolonet4_640, yolo_test_loader_640, image_size=640)
del yolonet4_640
gc.collect()





In [ ]:
assert len(detection0_640) == len_test
assert len(detection1_640) == len_test
assert len(detection2_640) == len_test
assert len(detection3_640) == len_test
assert len(detection4_640) == len_test


# VFNet

In [ ]:
import sys
import os
sys.path.append('/kaggle/working/mmdetection')
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check MMDetection installation
from mmdet.apis import set_random_seed

# Imports
import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

import random
import numpy as np
from pathlib import Path

In [ ]:
global_seed = 111

def set_seed(seed=global_seed):
    """Sets the random seeds."""
    set_random_seed(seed, deterministic=False)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

In [ ]:
os.listdir("/kaggle/working/mmdetection")

In [ ]:
from mmcv import Config
cfg = Config.fromfile('/kaggle/working/mmdetection/configs/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco.py')

In [ ]:
import datetime
now = datetime.datetime.now()

data = dict(
    info=dict(
        description=None,
        url=None,
        version=None,
        year=now.year,
        contributor=None,
        date_created=now.strftime('%Y-%m-%d %H:%M:%S.%f'),
    ),
    licenses=[dict(
        url=None,
        id=0,
        name=None,
    )],
    images=[
        # license, url, file_name, height, width, date_captured, id
    ],
    type='instances',
    annotations=[
        # segmentation, area, iscrowd, image_id, bbox, category_id, id
    ],
    categories=[
        # supercategory, id, name
    ],
)
labels =  [
            "__ignore__",
            "opacity",
            ]
class_name_to_id = {}
for i, each_label in enumerate(labels):
    class_id = i - 1  # starts with -1
    class_name = each_label
    if class_id == -1:
        assert class_name == '__ignore__'
        continue
    class_name_to_id[class_name] = class_id
    data['categories'].append(dict(
        supercategory=None,
        id=class_id,
        name=class_name,
    ))

In [ ]:
from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, init_detector, inference_detector

In [ ]:
cfg.dataset_type = 'CocoDataset' # Dataset type, this will be used to define the dataset
cfg.classes = ("opacity",)

In [ ]:
from mmdet.apis import set_random_seed

# Imports
import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

import mmcv
from mmcv import Config
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmdet.apis import single_gpu_test
from mmdet.datasets import build_dataloader, build_dataset

In [ ]:
import cv2
import matplotlib.pyplot as plt

label2color = [[59, 238, 119]]

viz_labels =  ["Opacity"]

In [ ]:
import glob
WEIGHTS_FILE_LIST = glob.glob("../input/rerenovated-vfnet-pretrained/*pth")
options = dict(classes = ("opacity"))
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
import glob

import datetime
now = datetime.datetime.now()

data = dict(
    info=dict(
        description=None,
        url=None,
        version=None,
        year=now.year,
        contributor=None,
        date_created=now.strftime('%Y-%m-%d %H:%M:%S.%f'),
    ),
    licenses=[dict(
        url=None,
        id=0,
        name=None,
    )],
    images=[
        # license, url, file_name, height, width, date_captured, id
    ],
    type='instances',
    annotations=[
        # segmentation, area, iscrowd, image_id, bbox, category_id, id
    ],
    categories=[
        # supercategory, id, name
    ],
)
labels =  [
            "__ignore__",
            "opacity",
            ]
class_name_to_id = {}
for i, each_label in enumerate(labels):
    class_id = i - 1  # starts with -1
    class_name = each_label
    if class_id == -1:
        assert class_name == '__ignore__'
        continue
    class_name_to_id[class_name] = class_id
    data['categories'].append(dict(
        supercategory=None,
        id=class_id,
        name=class_name,
    ))
    


In [ ]:
import json
import shutil
from pathlib import Path
data_test = data.copy()
data_test['images'] = []
data_test['annotations'] = []

def get_test_annotation(image_paths, anno_name):
        #test_output_dir = f"train_{IMAGE_SIZE}"
#test_annotations = df_test.copy()
        for i, path in tqdm(enumerate(image_paths)):  
            #image_basename = Path(path).stem
            ## Copy Image

            ## Add Images to annotation
            #print(os.path.isfile(path))
            data_test['images'].append(dict(
                license = 0,
                url = None,
                file_name = path,
                height = 640,
                width = 640,
                data_captured = None,
                id = i

            ))
        with open(anno_name, 'w') as f:
            json.dump(data_test, f, indent=4)

In [ ]:
df_test['image_path'] = f'/kaggle/tmp/{split}/image/'+df_test['id']+".png"
df_test['image_path']

In [ ]:
vfnet_anno = ["vfnet_anno_test.json"]
test_annotation = get_test_annotation(df_test['image_path'].tolist(), vfnet_anno[0])
data_test['images'] = []

In [ ]:
DATASET_TYPE = 'CocoDataset'
cfg.data.test.img_prefix = ""
cfg.data.test.classes = ("opacity",)
cfg.data.test.ann_file = vfnet_anno[0]
cfg.data.test.type = DATASET_TYPE

In [ ]:
baseline_cfg_path = "/kaggle/working/mmdetection/configs/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco.py"
cfg = Config.fromfile(baseline_cfg_path)

cfg.classes = ("opacity")
cfg.data.test.img_prefix = ''
cfg.data.test.classes = cfg.classes


cfg.model.bbox_head.num_classes=1


cfg.seed = 211
set_random_seed(211, deterministic=False)
cfg.gpu_ids = [0]

cfg.data.test.pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=[(640,640)],
                flip=True,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip', direction='horizontal'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ]

cfg.test_pipeline = [
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=[(640,640)],
                flip=True,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip', direction='horizontal'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ]

cfg.model.test_cfg.score_thr = 0.001

In [ ]:
def format_pred(boxes: np.ndarray, scores: np.ndarray, labels: np.ndarray) -> str:
    pred_strings = []
    label_str = ['opacity']
    for label, score, bbox in zip(labels, scores, boxes):
        xmin, ymin, xmax, ymax = bbox.astype(np.int64)
        pred_strings.append(f"{label_str[int(label)]} {score:.16f} {xmin} {ymin} {xmax} {ymax}")
    return " ".join(pred_strings)


In [ ]:
total_results = []
total_image_infos = []
iou_nms = 0.5
device = 'cuda'
for fold in [0,1,2,3,4]:
        cfg.data.test.ann_file = f"vfnet_anno_test.json"
        viz_images = []
        results = []
        image_infos = []

        model = init_detector(cfg, WEIGHTS_FILE_LIST[fold])
        model.to(device)
        model.eval()

        with torch.no_grad():
            for index, row in tqdm(df_test.iterrows(), total=df_test.shape[0], position=0, leave=True):
                #original_H, original_W = (int(row.height), int(row.width))
                predictions = inference_detector(model, row.image_path)

                boxes = predictions[0][:,:4]

                scores = predictions[0][:,4]

                boxes = boxes / 640


                skip_box_thr = 0.0001
                iou_thr = iou_nms
                boxes_list = []
                scores_list = []
                weights = [1]
                labels_list = [[1]*boxes.shape[0]]


                boxes_list.append(boxes.tolist())
                scores_list.append(scores.tolist())
                boxes, scores, labels = nms(boxes_list,
                                                              scores_list,
                                                              labels_list,
                                                              weights=weights,
                                                              iou_thr = iou_thr,)
                                                          #skip_box_thr = skip_box_thr
                processed_det0 = np.zeros(shape=(len(boxes), 5))
                processed_det0[:,:4] = np.array(boxes)
                processed_det0[:,4] = np.array(scores)
                if processed_det0[0,0] > 2:
                    print('error')
                    break

                results.append(processed_det0)
                image_infos.append(row.id)
            total_results.append(results)
            total_image_infos.append(image_infos)
            gc.collect()



# CasCade

In [ ]:
WEIGHTS_FILE_LIST = glob.glob("../input/covid-cascade/*pth")
WEIGHTS_FILE_LIST

# CASCADE CFG

In [ ]:
cfg = Config.fromfile('/kaggle/working/mmdetection/configs/cascade_rcnn/cascade_rcnn_x101_32x4d_fpn_1x_coco.py')
for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 1
cfg.dataset_type = 'CocoDataset' # Dataset type, this will be used to define the dataset
cfg.classes = ("opacity",)
DATASET_TYPE = 'CocoDataset'
cfg.data.test.img_prefix = ""
cfg.data.test.classes = ("opacity",)
cfg.data.test.ann_file = vfnet_anno[0]
cfg.data.test.type = DATASET_TYPE
cfg.classes = ("opacity")
cfg.data.test.classes = cfg.classes

# cfg.model.roi_head.bbox_head.num_classes = 1
# cfg.model.bbox_head.num_classes = 1
#for head in cfg.model.roi_head.bbox_head:
#    head.num_classes = 1
#cfg.model.bbox_head.num_classes=1

# Set seed thus the results are more reproducible
cfg.seed = 211
set_random_seed(211, deterministic=False)
cfg.gpu_ids = [0]

cfg.data.test.pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=[(640,640)],
                flip=True,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip', direction='horizontal'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ]

cfg.test_pipeline = [
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=[(640,640)],
                flip=True,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip', direction='horizontal'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ]

# cfg.data.samples_per_gpu = 4
# cfg.data.workers_per_gpu = 4
# cfg.model.test_cfg.nms.iou_threshold = 0.3
cfg.model.test_cfg.score_thr = 0.001
cfg.model.test_cfg.rcnn.score_thr = 0.001
cfg.model.test_cfg.rcnn.nms.iou_threshold=1.1

In [ ]:
total_results_cascade = []
total_image_infos = []
iou_nms = 0.5
device = 'cuda'
for fold in [0,1,2,3,4]:
        cfg.data.test.ann_file = f"vfnet_anno_test.json"
        viz_images = []
        results = []
        image_infos = []

        model = init_detector(cfg, WEIGHTS_FILE_LIST[fold])
        model.to(device)
        model.eval()

        with torch.no_grad():
            for index, row in tqdm(df_test.iterrows(), total=df_test.shape[0], position=0, leave=True):
                #original_H, original_W = (int(row.height), int(row.width))
                predictions = inference_detector(model, row.image_path)

                boxes = predictions[0][:,:4]

                scores = predictions[0][:,4]

                boxes = boxes / 640


                skip_box_thr = 0.0001
                iou_thr = iou_nms
                boxes_list = []
                scores_list = []
                weights = [1]
                labels_list = [[1]*boxes.shape[0]]


                boxes_list.append(boxes.tolist())
                scores_list.append(scores.tolist())
                boxes, scores, labels = nms(boxes_list,
                                                              scores_list,
                                                              labels_list,
                                                              weights=weights,
                                                              iou_thr = iou_thr,)
                                                          #skip_box_thr = skip_box_thr
                processed_det0 = np.zeros(shape=(len(boxes), 5))
                processed_det0[:,:4] = np.array(boxes)
                processed_det0[:,4] = np.array(scores)
                if processed_det0[0,0] > 2:
                    print('error')
                    break

                results.append(processed_det0)
                image_infos.append(row.id)
            total_results_cascade.append(results)
            total_image_infos.append(image_infos)
            gc.collect()

# Weighted Boxes Fusion

In [ ]:
iou_thr = 0.6
skip_box_thr = 0.1
image_size = 640

#total_boxes = []
#total_scores = []
#total_labels = []

ensemble_detection = []

for i, (det0_640, det1_640, det2_640, det3_640, det4_640,
        vfnet0,vfnet1,vfnet2,vfnet3,vfnet4,
        cascade0,cascade1,cascade2,cascade3,cascade4) in enumerate(zip(detection0_640, detection1_640, detection2_640,\
                                                                            detection3_640, detection4_640,\
                                                                            total_results[0],total_results[1],\
                                                                            total_results[2],total_results[3],\
                                                                            total_results[4],
                                                                            total_results_cascade[0],\
                                                                            total_results_cascade[1],\
                                                                            total_results_cascade[2],\
                                                                            total_results_cascade[3],\
                                                                            total_results_cascade[4])):
                    #print(f"i : f {i}"+" ="*50)
    
    boxes_list = []
    scores_list = []
    weights=[1,1,1,1,1,
             1,1,1,1,1,
             1,1,1,1,1]
    
    labels_list = [[1]*len(det0_640),[1]*len(det1_640),[1]*len(det2_640),\
                   [1]*len(det3_640),[1]*len(det4_640),
                   [1]*len(vfnet0),[1]*len(vfnet1),[1]*len(vfnet2),[1]*len(vfnet3),[1]*len(vfnet4),
                   [1]*len(cascade0),[1]*len(cascade1),[1]*len(cascade2),[1]*len(cascade3),[1]*len(cascade4)]
    
    box0_640 = (det0_640[:,:4]).tolist()
    box1_640 = (det1_640[:,:4]).tolist()
    box2_640 = (det2_640[:,:4]).tolist()
    box3_640 = (det3_640[:,:4]).tolist()
    box4_640 = (det4_640[:,:4]).tolist()

    box0_vfnet = (vfnet0[:,:4]).tolist()
    box1_vfnet = (vfnet1[:,:4]).tolist()
    box2_vfnet = (vfnet2[:,:4]).tolist()
    box3_vfnet = (vfnet3[:,:4]).tolist()
    box4_vfnet = (vfnet4[:,:4]).tolist()
    
    box0_cascade = (cascade0[:,:4]).tolist()
    box1_cascade = (cascade1[:,:4]).tolist()
    box2_cascade = (cascade2[:,:4]).tolist()
    box3_cascade = (cascade3[:,:4]).tolist()
    box4_cascade = (cascade4[:,:4]).tolist()
    
   
    
    conf0_640 = det0_640[:,4].tolist()
    conf1_640 = det1_640[:,4].tolist()
    conf2_640 = det2_640[:,4].tolist()
    conf3_640 = det3_640[:,4].tolist()
    conf4_640 = det4_640[:,4].tolist()

    conf0_vfnet = vfnet0[:,4].tolist()
    conf1_vfnet = vfnet1[:,4].tolist()
    conf2_vfnet = vfnet2[:,4].tolist()
    conf3_vfnet = vfnet3[:,4].tolist()
    conf4_vfnet = vfnet4[:,4].tolist()
    
    conf0_cascade = cascade0[:,4].tolist()
    conf1_cascade = cascade1[:,4].tolist()
    conf2_cascade = cascade2[:,4].tolist()
    conf3_cascade = cascade3[:,4].tolist()
    conf4_cascade = cascade4[:,4].tolist()
    
    
    boxes_list.append(box0_640)
    boxes_list.append(box1_640)
    boxes_list.append(box2_640)
    boxes_list.append(box3_640)
    boxes_list.append(box4_640)

    boxes_list.append(box0_vfnet)
    boxes_list.append(box1_vfnet)
    boxes_list.append(box2_vfnet)
    boxes_list.append(box3_vfnet)
    boxes_list.append(box4_vfnet)
    
    boxes_list.append(box0_cascade)
    boxes_list.append(box1_cascade)
    boxes_list.append(box2_cascade)
    boxes_list.append(box3_cascade)
    boxes_list.append(box4_cascade)

    
    scores_list.append(conf0_640)
    scores_list.append(conf1_640)
    scores_list.append(conf2_640)
    scores_list.append(conf3_640)
    scores_list.append(conf4_640)

    scores_list.append(conf0_vfnet)
    scores_list.append(conf1_vfnet)
    scores_list.append(conf2_vfnet)
    scores_list.append(conf3_vfnet)
    scores_list.append(conf4_vfnet)
    
    scores_list.append(conf0_cascade)
    scores_list.append(conf1_cascade)
    scores_list.append(conf2_cascade)
    scores_list.append(conf3_cascade)
    scores_list.append(conf4_cascade)

    
    boxes, scores, labels = weighted_boxes_fusion(boxes_list,
                                                  scores_list,
                                                  labels_list,
                                                  weights=weights,
                                                  iou_thr = iou_thr,
                                                  skip_box_thr = skip_box_thr)
    

    processed_det0 = np.zeros(shape=(len(boxes), 5))
    processed_det0[:,:4] = np.array(boxes)*image_size
    processed_det0[:,4] = np.array(scores)
    ensemble_detection.append(processed_det0)

In [ ]:
def make_df_image(df_valid, detection):
    df_image = pd.DataFrame()
    df_image.loc[:,'id'] = df_valid.image + '_image'

    predict_string = []
    for i,det in enumerate(detection):
        d = df_valid.iloc[i]

        s = ''
        for x0, y0, x1, y1, c in det:
            x0 = int(x0/image_size*d.dim1 )
            y0 = int(y0/image_size*d.dim0)
            x1 = int(x1/image_size*d.dim1 )
            y1 = int(y1/image_size*d.dim0)
            s += ' opacity %0.5f %4d %4d %4d %4d'%(c,x0,y0,x1,y1)
        predict_string.append(s)

    df_image.loc[:, 'PredictionString'] = predict_string
    return df_image

In [ ]:
pred_df = make_df_image(df_test, ensemble_detection)

In [ ]:
test_df = test_df.drop(['PredictionString'], axis=1)
sub_df = pd.merge(test_df, pred_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")

In [ ]:
sub_df = sub_df[['id', 'PredictionString']]

In [ ]:
sub_df['none'] = df_2class['none']
for i in range(sub_df.shape[0]):
    if sub_df.loc[i,'PredictionString'] == "none 1 0 0 1 1":
        continue
    sub_df_split = sub_df.loc[i,'PredictionString'].split()
    twoclass_none = sub_df.loc[i,'none']
    sub_df_list = []
    final_sub_df_list = []
    for j in range(int(len(sub_df_split) / 6)):
        sub_df_list.append(['opacity',
                                        (float(sub_df_split[6 * j + 1]))*(1-twoclass_none)**0.20,
                                        sub_df_split[6 * j + 2],
                                        sub_df_split[6 * j + 3],
                                        sub_df_split[6 * j + 4],
                                        sub_df_split[6 * j + 5]]
                                    )
    sub_df_list = sorted(sub_df_list, key = lambda x: x[1], reverse=True)
    for zz in range(len(sub_df_list)):
        sub_df_list[zz][1] = str(sub_df_list[zz][1])
        final_sub_df_list.extend(sub_df_list[zz])

    sub_df.loc[i,'PredictionString'] = ' '.join(final_sub_df_list)

for i in range(sub_df.shape[0]):
    if sub_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
        sub_df.loc[i,'PredictionString'] = sub_df.loc[i,'PredictionString'] + ' none ' + str(sub_df.loc[i,'none']) + ' 0 0 1 1'
sub_df = sub_df[['id', 'PredictionString']]   
df_study = df_study[:study_len]
df_study = df_study.append(sub_df).reset_index(drop=True)
df_study.to_csv('/kaggle/working/submission.csv',index = False)

In [ ]:
!rm -r /kaggle/working/mmdetection